<a href="https://colab.research.google.com/github/rajivsam/example_ITSM/blob/main/notebooks/colab_version_generate_report_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/rajivsam/example_ITSM.git

In [ ]:
import os
os.chdir('example_ITSM/notebooks')

In [ ]:
! pip install -r ../requirements.txt

In [ ]:
import pandas as pd
fp = "../data/ticket_resolution_Q2_2016.csv"
df = pd.read_csv(fp)

In [ ]:
fpdtypes = "../data/data_types.csv"
dtypes_df = pd.read_csv(fpdtypes)
dtypes_dict = {row["attribute"]: row["type"] for index, row in dtypes_df.iterrows()}
df = df.astype(dtypes_dict)

In [ ]:
cols_closed = ["number", "sys_created_at", "incident_state", "assignment_group", "closed_at"]
df_ct = df[cols_closed].copy()
del df

In [ ]:
selection_closed = (df_ct["incident_state"] == "Closed" )

In [ ]:
df_ct["resolution_time"] = df_ct["closed_at"] - df_ct["sys_created_at"]

In [ ]:
df_ct["resolution_time"] = df_ct["resolution_time"].apply(lambda x: x.total_seconds()/(3600))

In [ ]:
selection_valid_resolution_time = df_ct["resolution_time"] > 0

In [ ]:
valid_records = selection_closed & selection_valid_resolution_time
df_ct = df_ct[valid_records]

In [ ]:
fp_q2_2016 = "../data/plot_data_q2_2016_ticket_activity.csv"
df_ct.to_csv(fp_q2_2016, index=False)
select_group_10_closing_activity = df_ct["assignment_group"] == "Group 10"
df_ct[select_group_10_closing_activity]

In [ ]:
import plotly.express as px
import numpy as np

fig = px.histogram(df_ct, x="resolution_time", labels=dict(resolution_time="resolution time in hours"))
fig.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np
np.random.seed(1)
fig = px.violin(df_ct, y="resolution_time", box=True, points="all", hover_data=df_ct.columns)
fig.show()

In [ ]:
import plotly.express as px
fig = px.ecdf(df_ct, x="resolution_time")
fig.show()